In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import warnings
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
import pyarrow.parquet as pq
from surprise import SVD
from surprise import accuracy
import pickle

In [29]:
games=pd.read_parquet("games.parquet")
#items=pd.read_parquet("items.parquet")
reviews=pd.read_parquet("reviews.parquet")

In [30]:
merged_df = pd.merge(reviews[['item_id', 'user_id', 'sentiment_analysis',"recommend"]], games[['app_name',"item_id"]], left_on='item_id', right_on="item_id")

In [31]:
merged_df.shape

(44739, 5)

In [32]:
merged_df['recommend'] = merged_df['recommend'].replace({True: 1, False: 0})

In [78]:
for index, row in merged_df.iterrows():
    if (row[2] == 2) and (row[3] == 1):
        merged_df.at[index, 'rating'] = 5
    elif (row[2] == 2) and (row[3] == 0):
        merged_df.at[index, 'rating'] = 4
    elif (row[2] == 1) and (row[3] == 1):
        merged_df.at[index, 'rating'] = 3
    elif (row[2] == 1) and (row[3] == 0):
        merged_df.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 1):
        merged_df.at[index, 'rating'] = 1
    elif (row[2] == 0) and (row[3] == 0):
        merged_df.at[index, 'rating'] = 0

C:\Users\Vivobook\AppData\Local\Temp\ipykernel_12160\41572838.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (row[2] == 2) and (row[3] == 1):
C:\Users\Vivobook\AppData\Local\Temp\ipykernel_12160\41572838.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (row[2] == 2) and (row[3] == 0):
C:\Users\Vivobook\AppData\Local\Temp\ipykernel_12160\41572838.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (row[2] == 1) a

In [34]:
merged_df

,item_id,user_id,sentiment_analysis,recommend,app_name,rating
0,1250,76561197970982479,2,1,Killing Floor,5.0
1,1250,death-hunter,2,1,Killing Floor,5.0
2,1250,DJKamBer,0,1,Killing Floor,1.0
3,1250,diego9031,1,1,Killing Floor,3.0
4,1250,76561198081962345,1,1,Killing Floor,3.0
...,...,...,...,...,...,...
44734,367780,laislabonita75,2,1,Aero's Quest,5.0
44735,367780,evilindiegaming,2,1,Aero's Quest,5.0
44736,305920,laislabonita75,0,0,Another Perspective,0.0
44737,306040,Gamer0009,2,1,The Howler,5.0


In [35]:
merged_df.to_parquet('modelo.parquet')

c:\Users\Vivobook\miniconda3\envs\datascience\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [36]:
# Leer el archivo .parquet y cargarlo en un DataFrame de pandas
new_df = pd.read_parquet('modelo.parquet')

In [11]:
new_df

,item_id,user_id,sentiment_analysis,recommend,app_name,rating
0,1250,76561197970982479,2,1,Killing Floor,5.0
1,1250,death-hunter,2,1,Killing Floor,5.0
2,1250,DJKamBer,0,1,Killing Floor,1.0
3,1250,diego9031,1,1,Killing Floor,3.0
4,1250,76561198081962345,1,1,Killing Floor,3.0
...,...,...,...,...,...,...
44734,367780,laislabonita75,2,1,Aero's Quest,5.0
44735,367780,evilindiegaming,2,1,Aero's Quest,5.0
44736,305920,laislabonita75,0,0,Another Perspective,0.0
44737,306040,Gamer0009,2,1,The Howler,5.0


In [12]:
resultados = new_df[new_df['user_id'] == 'death-hunter']
resultados

,item_id,user_id,sentiment_analysis,recommend,app_name,rating
1,1250,death-hunter,2,1,Killing Floor,5.0
9812,4000,death-hunter,2,1,Garry's Mod,5.0
11464,232090,death-hunter,2,1,Killing Floor 2,5.0


In [13]:
reader = Reader(rating_scale=(0, 5))  # Adaptar rating_scale a tus datos

In [14]:
data = Dataset.load_from_df(new_df[['user_id', 'item_id', 'rating']], reader)

In [15]:
#Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2)  # Puedes ajustar el tamaño de prueba

In [16]:
#optimizacion de hiperparametros 
from surprise.model_selection import cross_validate

rmse_test_means = []
factores = [1,2,4,8,16,32,64,128,256]

for factor in factores:
    print(factor)
    model = SVD(n_factors=factor)
    cv = cross_validate(model, data, measures=['RMSE'], cv = 3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

1
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.5604  1.5550  1.5503  1.5552  0.0041  
Fit time          0.19    0.18    0.20    0.19    0.01    
Test time         0.08    0.07    0.12    0.09    0.02    
2
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.5570  1.5542  1.5546  1.5553  0.0012  
Fit time          0.18    0.18    0.22    0.19    0.02    
Test time         0.09    0.07    0.08    0.08    0.01    
4
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.5481  1.5656  1.5589  1.5575  0.0072  
Fit time          0.27    0.19    0.22    0.23    0.03    
Test time         0.22    0.10    0.17    0.16    0.05    
8
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.5644  1.5

In [17]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors': [5,50,100],'n_epochs': [5, 10,20], 'lr_all': [0.001, 0.002, 0.005],
              'reg_all': [0.002, 0.02, 0.2]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs = -1)
gs.fit(data)

In [18]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.5524090410729185
{'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}


In [19]:
best_model=gs.best_params

In [20]:
#Seleccionar un algoritmo (por ejemplo, SVD) y entrenar el modelo
model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.2)
model.fit(trainset)

In [22]:
with open('modelo.pkl', 'wb') as archivo:
    pickle.dump(model, archivo)

In [23]:
with open ('modelo.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)


In [24]:
print(modelo)

In [79]:
def recomendacion_usuario( id_usuario ):
    #ID del usuario para el cual quieres obtener recomendaciones
    usuario_especifico = id_usuario  # Reemplaza 'js41637' con el ID del usuario para el que quieres las recomendaciones

    #Crear una lista de juegos ya valorados por el usuario específico
    juegos_valorados = merged_df[merged_df['user_id'] == usuario_especifico]['app_name'].unique()

    #Crear una lista de todos los juegos disponibles
    todos_los_juegos = merged_df['app_name'].unique()

    #Crear una lista de juegos no valorados por el usuario específico
    juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

    #Generar predicciones para los juegos no valorados por el usuario
    predicciones = [model.predict(usuario_especifico, juego) for juego in juegos_no_valorados]

    #Ordenar las predicciones en base a la valoración y obtener los juegos recomendados
    recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5]  # Obtener las 5 mejores recomendaciones

    #Mostrar los juegos recomendados
   
    lista=list()
    for recomendacion in recomendaciones:
        lista.append(recomendacion.iid)
    return f'Juego1: {lista[0]}, Juego2: {lista[1]}, Juego3: {lista[2]}, Juego4: {lista[3]}, Juego5: {lista[4]}'
    

In [80]:
recomendacion_usuario('death-hunter')

'Juego1: Thief™ Gold, Juego2: The Dream Machine: Chapter 1 & 2, Juego3: Heroes of Might & Magic V, Juego4: S.T.A.L.K.E.R.: Shadow of Chernobyl, Juego5: Splice'

In [84]:
#model.fit(trainset)

#Realizar predicciones en el conjunto de prueba
predictions = model.test(testset)

    #Evaluar las predicciones
accuracy.rmse(predictions)
    #ID del usuario para el cual quieres obtener recomendaciones
usuario_especifico = "JunG00berfuckingsucks"  # Reemplaza 'js41637' con el ID del usuario para el que quieres las recomendaciones

    #Crear una lista de juegos ya valorados por el usuario específico
juegos_valorados = merged_df[merged_df['user_id'] == usuario_especifico]['app_name'].unique()

    #Crear una lista de todos los juegos disponibles
todos_los_juegos = merged_df['app_name'].unique()

    #Crear una lista de juegos no valorados por el usuario específico
juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

    #Generar predicciones para los juegos no valorados por el usuario
predicciones = [model.predict(usuario_especifico, juego) for juego in juegos_no_valorados]

    #Ordenar las predicciones en base a la valoración y obtener los juegos recomendados
recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5]  # Obtener las 5 mejores recomendaciones

    #Mostrar los juegos recomendados
for recomendacion in recomendaciones:
    print (f"Juego: {recomendacion.iid}, Valoración estimada: {recomendacion.est}")
 

RMSE: 1.5526
Juego: Thief™ Gold, Valoración estimada: 3.899197787482235
Juego: The Dream Machine: Chapter 1 & 2, Valoración estimada: 3.899197787482235
Juego: Heroes of Might & Magic V, Valoración estimada: 3.899197787482235
Juego: S.T.A.L.K.E.R.: Shadow of Chernobyl, Valoración estimada: 3.899197787482235
Juego: Splice, Valoración estimada: 3.899197787482235


In [85]:
recomendacion_usuario('death-hunter')

'Juego1: Thief™ Gold, Juego2: The Dream Machine: Chapter 1 & 2, Juego3: Heroes of Might & Magic V, Juego4: S.T.A.L.K.E.R.: Shadow of Chernobyl, Juego5: Splice'

In [87]:
recomendacion_usuario('Gamer0009')

'Juego1: Thief™ Gold, Juego2: The Dream Machine: Chapter 1 & 2, Juego3: Heroes of Might & Magic V, Juego4: S.T.A.L.K.E.R.: Shadow of Chernobyl, Juego5: Splice'